## FAISS - facebook AI similarity search

In [99]:
#Install Packages
# %pip install faiss-cpu
# %pip install sentence-transformers
# %pip install sentence-transformers

In [100]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [101]:
df = pd.read_csv("sample_text.csv")
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


### Step 1 : Create source embeddings for the text column

In [102]:
# Install required packages (run once in the notebook)
# %pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)
vectors.shape

(8, 768)

In [103]:
dim = vectors.shape[1]

#### Step 2 : Build a FAISS Index for vectors

In [104]:
import faiss

index = faiss.IndexFlatL2(dim)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001EFE105F600> >

#### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [105]:
index.add(vectors)

In [106]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001EFE105F600> >

#### Step 4 : Encode search text using same encorder and normalize the output vector

In [107]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [108]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [109]:
distances, I = index.search(svec, k=2)
I

array([[3, 2]])

In [110]:
df.loc[I[0]]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion
